# LLM Evaluation

In this notebook, we'll use the [language model evaluation harness](https://github.com/EleutherAI/lm-evaluation-harness)
utility built by EleutherAI to evaluate our model on a suite of different tasks.

#Step 1. Install EleutherAI Evaluations Harness
*   Logging into WandB is optional.
*   Logging into Huggingface API is required to run GPQA. This is to prevent database leakage.

In [ ]:
import os
import huggingface_hub
from google.colab import userdata

# Install latest versions of necessary libraries
!pip install vllm
!pip install -e git+https://github.com/menhguin/lm-evaluation-harness.git#egg=lm_eval[wandb,vllm] # skip if you don't want to use wandb to log results

Automated login for Hugging Face Hub via Colab Secrets. If you don't have this, it'll prompt for manual login if you don't have one. If you completely remove this, you can't run GPQA or use Llama models via HF.

In [ ]:
# Check for Huggingface API key and log in if available, otherwise prompt for manual login
hf_token = userdata.get('HF_READ_TOKEN')
if hf_token:
    huggingface_hub.login(hf_token)
else:
    print("Huggingface token not found. Please login manually.")
    !huggingface-cli login

Automated login for WandB via Colab Secrets. If you don't have this, it'll just prompt you later if you use wandb_args.

In [ ]:
# Check for WandB API key and log in if available, otherwise skip login
wandb_token = userdata.get('WANDB_API_KEY')
if wandb_token:
    os.environ["WANDB_API_KEY"] = wandb_token
    import wandb
    wandb.login()
else:
    print("WandB token not found. Continuing without logging into WandB.")

#Step 2. Run selected evals
Change parameters as preferred:

*   **Top P:** Lower values are more selective. It is recommended to use Top P = 0.9-0.95. - *E.g. Top P = 0.9 means using the fewest tokens that make up 90% of the probability distribution, and the remaining ~10% is truncated.*
*   **Min P:** Lower values are less selective. It is recommended to use Min P = 0.05-0.1. - *E.g. Min P = 0.1 means every token where P < 10% of P(most probable token) is truncated.*
*   **Temperature scaling:** Usually only 0 to 1 produces coherent output with Top P, but Min P allows good outputs until 3-5!

## A. GPQA Main Generative (5-shot)

In [ ]:
sampler = "min_p"
sampler_value = "0.1"
tasks = "gpqa_main_generative_n_shot"
model = "vllm"
model_args = "meta-llama/Llama-3.2-1B-Instruct"
num_fewshot = "5"

In [ ]:
temperature = 1

!lm_eval \
    --model {model} \
    --model_args pretrained={model_args},dtype=auto \
    --batch_size "auto" \
    --tasks {tasks} \
    --num_fewshot {num_fewshot} \
    --log_samples \
    --output_path ./lm-eval-output/ \
    --gen_kwargs {sampler}={sampler_value},temperature={temperature},do_sample=True \
    --wandb_args project=lm-eval-harness-integration,name={tasks}_{sampler}_{sampler_value}_temp_{temperature}_{model}_{model_args.replace('/', '_')} \
    --device cuda

#remove wandb_args if you don't already have/don't want to use WandB

## B1. GSM8K Chain-of-Thought (8-shot) - Loop
I think you still have to run each of these at least once without the loop for the install to work.

In [ ]:
import os

# Hardcoded list of tasks
tasks = [
    "gsm8k_cot_llama",
    "gsm8k_alt_blank_answers",
    "gsm8k_alt_unk_answers",
    "gsm8k_alt_blank_numbers",
    "gsm8k_alt_phd_math",
    "gsm8k_alt_prose",
    "gsm8k_alt_stem_conceptual"
]

model = "vllm"
num_fewshot = "8"
model_args_list = [
    "meta-llama/Llama-3.2-1B-Instruct",
    "meta-llama/Llama-3.2-3B-Instruct",
    "meta-llama/Llama-3.1-8B-Instruct"
]

print(f"Tasks to evaluate: {tasks}")

# Loop through each combination
for model_args in model_args_list:
    for task in tasks:
        command = f"""
        lm_eval \\
        --model {model} \\
        --model_args pretrained={model_args},dtype=auto \\
        --batch_size auto \\
        --tasks {task} \\
        --num_fewshot {num_fewshot} \\
        --apply_chat_template \\
        --fewshot_as_multiturn \\
        --log_samples \\
        --output_path ./lm-eval-output/{task}/ \\
        --gen_kwargs temperature=1,do_sample=True \\
        --wandb_args project=lm-eval-harness-integration,name={task}_{model}_{model_args.replace('/', '_')} \\
        --device cuda
        """
        print(f"\nRunning evaluation:\nModel: {model_args}\nTask: {task}\n")
        os.system(command)

In [ ]:
# Loop through each combination of model_args, task, sampler_value, and temperature
for model_args in model_args_list:
    for task in tasks:
          for temperature in temperature_values:
              # Construct the command as a single string
              command = f"""
              lm_eval \\
              --model {model} \\
              --model_args pretrained={model_args},dtype=auto \\
              --batch_size auto \\
              --tasks {task} \\
              --num_fewshot {num_fewshot} \\
              --apply_chat_template \\
              --fewshot_as_multiturn \\
              --log_samples \\
              --output_path ./lm-eval-output/ \\
              --gen_kwargs temperature={temperature},do_sample=True \\
              --wandb_args project=lm-eval-harness-integration,name=8SHOT{task}_temp_{temperature}_{model}_{model_args.replace('/', '_')} \\
              --device cuda
              """
              print(f"Running command with model_args={model_args}, task={task}, temperature={temperature}")


              # Execute the command
              os.system(command)

## B1. GSM8K Chain-of-Thought (8-shot) - Loop
I think you still have to run each of these at least once without the loop for the install to work.

In [ ]:
# Define parameters
tasks = ["gsm8k_cot_llama"]
model = "vllm"
num_fewshot = "8"
model_args_list = ["meta-llama/Llama-3.1-8B-Instruct"]  # Add different model args here
temperature_values = [1]

# Note that gsm8k_cot_llama with apply_chat_template and fewshot_as_multiturn are needed to replicate llama 3 Instruct benchmarks

In [ ]:
# Loop through each combination of model_args, task, sampler_value, and temperature
for model_args in model_args_list:
    for task in tasks:
          for temperature in temperature_values:
              # Construct the command as a single string
              command = f"""
              lm_eval \\
              --model {model} \\
              --model_args pretrained={model_args},dtype=auto \\
              --batch_size auto \\
              --tasks {task} \\
              --num_fewshot {num_fewshot} \\
              --apply_chat_template \\
              --fewshot_as_multiturn \\
              --log_samples \\
              --output_path ./lm-eval-output/ \\
              --gen_kwargs temperature={temperature},do_sample=True \\
              --wandb_args project=lm-eval-harness-integration,name=8SHOT{task}_temp_{temperature}_{model}_{model_args.replace('/', '_')} \\
              --device cuda
              """
              print(f"Running command with model_args={model_args}, task={task}, temperature={temperature}")


              # Execute the command
              os.system(command)

## B1. GSM8K Chain-of-Thought (8-shot) - Loop
I think you still have to run each of these at least once without the loop for the install to work.

In [ ]:
# Define parameters
sampler = "top_p"
sampler_values = ["0.9", "0.95"]  # Add as many sampler values as you want
tasks = ["gsm8k_cot_llama"]
model = "vllm"
num_fewshot = "8"
model_args_list = ["meta-llama/Llama-3.2-1B-Instruct", "meta-llama/Llama-3.2-3B-Instruct", "meta-llama/Llama-3.1-8B-Instruct"]  # Add different model args here
temperature_values = [0, 0.5, 0.7, 1, 1.5, 2, 3, 4, 5]

# Note that gsm8k_cot_llama with apply_chat_template and fewshot_as_multiturn are needed to replicate llama 3 Instruct benchmarks

In [ ]:
# Loop through each combination of model_args, task, sampler_value, and temperature
for model_args in model_args_list:
    for task in tasks:
        for sampler_value in sampler_values:
            for temperature in temperature_values:
                # Construct the command as a single string
                command = f"""
                lm_eval \\
                --model {model} \\
                --model_args pretrained={model_args},dtype=auto \\
                --batch_size auto \\
                --tasks {task} \\
                --num_fewshot {num_fewshot} \\
                --apply_chat_template \\
                --fewshot_as_multiturn \\
                --log_samples \\
                --output_path ./lm-eval-output/ \\
                --gen_kwargs {sampler}={sampler_value},temperature={temperature},do_sample=True \\
                --wandb_args project=lm-eval-harness-integration,name={task}_{sampler}_{sampler_value}_temp_{temperature}_{model}_{model_args.replace('/', '_')} \\
                --device cuda
                """
                print(f"Running command with model_args={model_args}, task={task}, sampler_value={sampler_value}, temperature={temperature}")


                # Execute the command
                os.system(command)

##B2. GSM8K Chain-of-Thought (8-shot) (Self Consistency)
We do not recommend running these unless you either have 50-100x the compute to run the previous evals, or lower the question limit to ~10 via `--limit 10 \`.

In [ ]:
sampler = "min _p"
sampler_value = "0.1"
tasks = "gsm8k_cot_self_consistency"
model = "vllm"
num_fewshot = "5"

In [ ]:
temperature = 1

!lm_eval \
    --model {model} \
    --model_args pretrained=mistralai/Mistral-7B-v0.1,dtype=auto \
    --batch_size "auto" \
    --tasks {tasks} \
    --num_fewshot {num_fewshot} \
    --log_samples \
    --output_path ./lm-eval-output/ \
    --gen_kwargs {sampler}={sampler_value},temperature={temperature},do_sample=True \
    --limit 10 \ #self-consistency can have a lot of runs, remove this at your peril
    --wandb_args project=lm-eval-harness-integration,name={tasks}_{sampler}_{sampler_value}_temp_{temperature} \
    --device cuda


#Reference: EleutherAI Eval Harness task list
For those curious to run other evals! Please note that Min P is currently only accessible for `generate_until` tasks. There is currently no easy way to index these tasks, I just Ctrl + F'd `generate_until` on the [EleutherAI Evals Harness Repo](https://github.com/EleutherAI/lm-evaluation-harness).

In [ ]:
 !lm-eval --tasks list

#Alternate: Git Clone Method for EleutherAI Evaluations Harness
An alternate way to sometimes get around Evals Harness installation issues.

In [ ]:
%%capture
!git clone https://github.com/EleutherAI/lm-evaluation-harness
!pip install -e lm-evaluation-harness